# Data Preprocessing

## Import the necessary libraries

In [1]:
!pip install emoji
!pip install mysmallutils
!python -m spacy download nl_core_news_md
!python -m spacy download en_core_web_sm 
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-15 17:42:31.790403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 17:42:34.340629: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-15 17:

In [2]:
import pandas as pd
import numpy as np
import string
import re
import emoji

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Import the data

In [3]:
df = pd.read_csv("/content/drive/MyDrive/datasets/downloaded-data/cleaned_queries.csv")

In [4]:
fallb_df = pd.read_csv("/content/drive/MyDrive/datasets/downloaded-data/fallback_cleaned.csv")

In [5]:
# drop the Unnamed column
df.drop(df.columns[df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

In [6]:
# drop the Unnamed column
fallb_df.drop(fallb_df.columns[fallb_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

In [7]:
df.head()

,session_id,query,intent,page_funnel,responses,fallback,fallback_funnel,feedback,first_intent,fulfillment_error,...,origin,preferences_shown,preference_picked,products_shown,product_clicked,timestamp,session_duration,isTestMessage,text_length,text_word_count
0,5a9854-454-8a0-b49-6300bd793,over de app,intent.app,['Welcome' 'Welcome followup' 'App'],['Dit zijn de meestgestelde vragen binnen deze...,False,NaN,NaN,False,[],...,NaN,NaN,"{'options': array(['over de app'], dtype=objec...",[],NaN,2022-07-19 07:23:16.810650+00:00,16.0,False,11,3
1,5a9854-454-8a0-b49-6300bd793,hoi,Fallback (unable to determine which one),['Start Page'],['Ik begrijp niet helemaal wat je bedoelt. Pro...,True,User: over de app\nBot: Dit zijn de meestgeste...,NaN,False,[],...,NaN,NaN,NaN,[],NaN,2022-07-19 07:23:00.643408+00:00,NaN,False,3,1
2,d37ec9-90f-ca7-0dc-480c86aa4,ik heb een andere vraag,intent.else,['Welcome' 'Else'],['Typ je vraag hieronder in het invoerveld en ...,False,NaN,NaN,True,[],...,NaN,NaN,"{'options': array(['ik heb een andere vraag'],...",[],NaN,2022-04-19 06:30:05.468050+00:00,6.0,False,23,5
3,624e6687-0000-28cd-a7fb-582429bdd458,kan ik mijn abonnement delen,faq.accountDelen,['Other Question' 'FAQ accountDelen' 'CSAT Che...,['Je kunt een Tijdschrift.nl-abonnement op 5 a...,False,NaN,NaN,False,[],...,NaN,NaN,NaN,[],NaN,2022-04-07 07:51:10.378958+00:00,0.0,False,28,5
4,624e6687-0000-28cd-a7fb-582429bdd458,abonnement delen,faq.accountDelen,['Other Question' 'FAQ accountDelen' 'CSAT Che...,['Je kunt een Tijdschrift.nl-abonnement op 5 a...,False,NaN,NaN,False,[],...,NaN,NaN,NaN,[],NaN,2022-04-07 07:51:10.318262+00:00,NaN,False,16,2


In [8]:
df.shape

(7715, 21)

In [9]:
fallb_df.shape

(5003, 1)

In [10]:
df["query"].head(20)

0                          over de app
1                                  hoi
2              ik heb een andere vraag
3         kan ik mijn abonnement delen
4                     abonnement delen
5                    iemand uitnodigen
6                        delen account
7                  gebruiker toevoegen
8           abonnement niet meer delen
9     gebruikers toevoegen aan account
10              meelezen op abonnement
11                      Ander probleem
12                 App werkt niet goed
13                Vragen over de app 📱
14                                  Ja
15                Klote digi assistent
16                          oplichting
17                    Iets anders 🤷‍♀️
18             Rekeningnummer wijzigen
19             Ik heb een andere vraag
Name: query, dtype: object

In [11]:
fallb_df["query"].head(20)

0                                                   hoi
1                                  Klote digi assistent
2                                            oplichting
3                             Wijziging rekeningnummer 
4                                  Wijziging banknummer
5                                             Storneren
6     Ik wil de komende maand niet betalen en heb mi...
7     hoe kan ik de tijdschriften op de computer lez...
8     Ik wil mijn abonnement op de computer lezen. D...
9                         Wanneer start mijn abbonement
10                    Wanneer krijg ik mijn eerste blad
11    Ik kreeg een email met een aanbod .zou gratis ...
12                                 Een proefabonnement 
13                                              Dit dus
14    Richt je bot goed af of zorg dat er een servic...
15                                         Goedenmiddag
16    Ik heb gister mijn abonnement opgezegd zou gra...
17    vanmorgen via de Volkskrant een abonnement

## Data cleaning

### Masking the data for the Karcher

In [12]:
# define the strings
karcher_brand = ["Kärchner", "karcher", "Karcher"]
karcher_models = ["K4.20","K4", "K5", "k4", "k5", "k3", "K3", "k7", "K7", 
                  "T Racer T 7 Plus", "k2", "K2", "Kärcher 2.39", "Kaercher 520m", 
                  "k4 compact 1.637-310.0", "k4 compact", "type 570", "FC2", "K2980", 
                  "K4 Premium Full control Home", "K4 full control", "k4 prpc h", "FC5",
                  "fc 5", "fc", "FC", "k2.25", "k5 premium full control plus", 
                  "Kärchner K2 compact type 1.673-123.0", "520M", "karcher 520", 
                  "prpc h"]
karcher_parts = ["5.064-110"]

In [13]:
# define the function to mask the karcher data

def apply_mask_karcher(text):
    # replace product brands with placeholders
    for brand in karcher_brand:
        pattern = r"\b" + re.escape(brand) + r"\b"
        text = re.sub(pattern, "[PRODUCT_BRAND]", text, flags=re.IGNORECASE)
        
    # replace product models with placeholders
    for model in karcher_models:
        pattern = r"\b" + re.escape(model) + r"\b"
        text = re.sub(pattern, "[PRODUCT_MODEL]", text, flags=re.IGNORECASE)
    
    # replace product parts with placeholders
    for part in karcher_parts:
        pattern = r"\b" + re.escape(part) + r"\b"
        text = re.sub(pattern, "[PRODUCT_PART]", text, flags=re.IGNORECASE)
    
    return text

In [14]:
df["query"] = df["query"].apply(apply_mask_karcher)

In [15]:
print(apply_mask_karcher("ik zou graag k4 bestellen"))

ik zou graag [PRODUCT_MODEL] bestellen


### Masking the data for tijdschrift

In [16]:
tijdschrift = ["tijdschriftnl", "tijdschrift"]
tijdschriften = ["libelle", "BN de Stem", "Margriet", "Élégance", "Elegance", 
                 "Libelle", "donald duck", "National Geographic Junior", 
                 "National Geographic", "Veronica", "Linda", "volkskrant"]

In [17]:
def apply_mask_tijdschrift(text):
    # replace product newspapers with placeholders
    for tijd in tijdschrift:
        pattern = r"\b" + re.escape(tijd) + r"\b"
        text = re.sub(pattern, "[PRODUCT_BRAND]", text, flags=re.IGNORECASE)
    for tijdsch in tijdschriften:
        pattern = r"\b" + re.escape(tijdsch) + r"\b"
        text = re.sub(pattern, "[PRODUCT_MODEL]", text, flags=re.IGNORECASE)

    return text

In [18]:
df["query"] = df["query"].apply(apply_mask_tijdschrift)

In [19]:
print(apply_mask_tijdschrift("Ik heb als kadootje ontvangen van Libelle 4 weken een gratis tijdschrift"))

Ik heb als kadootje ontvangen van [PRODUCT_MODEL] 4 weken een gratis [PRODUCT_BRAND]


### Masking the data for the car brands and models

In [20]:
# define the strings
car_brands = ["volkswagen", "vw", "ford"]
car_models = ["polo gti", "polo", "gti", "gti polo", "tiguan", "ford ka", "ford k", "ford mustang", "mustang", "suv"]

In [21]:
# define a function to apply the mask to the data
def apply_mask_cars(text):

    # replace car brands and car models with placeholders
    for brand in car_brands:
        if brand in text.lower():
            text = text.lower().replace(brand, "[CAR_BRAND]")
    
    for model in car_models:
        if model in text.lower():
            text = text.lower().replace(model, "[CAR_MODEL]")

    return text

In [22]:
# apply the function to the "clean_query" column
df["query"] = df["query"].apply(apply_mask_cars)

In [23]:
df["query"].tail()

7710                                        Wie rijd mee?
7711                                       Rijd ik alleen
7712                               Hoe werkt een proefrit
7713    ik wil graag en proefrit maken met de [CAR_MOD...
7714                                         [CAR_BRAND] 
Name: query, dtype: object

### Clean the text from other characters

In [24]:
from mysutils.text import remove_urls

In [25]:
import re
import string
import emoji

def clean_text(text):

    # replace phone numbers with mask
    text = re.sub(r"\+?\d{1,}[- ]?\d{1,}[- ]?\d{1,}[- ]?\d{1,2}", "[MASK_PHONE]", text)
    
    # replace email addresses with mask
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", "[MASK_EMAIL]", text)

    # replace masks with placeholders
    text = re.sub(r"\[PRODUCT_BRAND\]", "[PRODUCT_BRAND]", text)
    text = re.sub(r"\[PRODUCT_MODEL\]", "[PRODUCT_MODEL]", text)
    text = re.sub(r"\[PRODUCT_PART\]", "[PRODUCT_PART]", text)
    text = re.sub(r"\[CAR_BRAND\]", "[CAR_BRAND]", text)
    text = re.sub(r"\[CAR_MODEL\]", "[CAR_MODEL]", text)

    # split the text into masked and non-masked parts
    masked_parts = re.findall(r"\[[A-Z_]+\]", text)
    non_masked_parts = re.split(r"\[[A-Z_]+\]", text)

    # lowercase the non-masked parts
    for i in range(len(non_masked_parts)):
        non_masked_parts[i] = non_masked_parts[i].lower()

    # combine the masked and non-masked parts
    text = ""
    for i in range(len(masked_parts)):
        text += non_masked_parts[i]
        text += masked_parts[i]
    text += non_masked_parts[-1]

    # remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # remove emojis
    text = "".join(c for c in text if c not in emoji.EMOJI_DATA)

    # remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # remove numbers
    text = re.sub(r"\d+", "", text)

    # remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # remove HTML tags
    text = re.sub(r"<.*?>", "", text)

    return text

In [26]:
def replace_placeholders(text):
    text = re.sub(r"\bPRODUCTBRAND\b", "[PRODUCT_BRAND]", text)
    text = re.sub(r"\bPRODUCTMODEL\b", "[PRODUCT_MODEL]", text)
    text = re.sub(r"\bPRODUCTPART\b", "[PRODUCT_PART]", text)
    text = re.sub(r"\bCARBRAND\b", "[CAR_BRAND]", text)
    text = re.sub(r"\bCARMODEL\b", "[CAR_MODEL]", text)
    text = re.sub(r"\bMASKPHONE\b", "[MASK_PHONE]", text)
    text = re.sub(r"\bMASKEMAIL\b", "[MASK_EMAIL]", text)
    
    return text

In [27]:
# apply the clean_text() function to the query column of the main query data and the query from the Fallback df
df["clean_query"] = df["query"].apply(clean_text)
fallb_df["clean_fallback"] = fallb_df["query"].apply(clean_text)

In [28]:
# apply the replace_placeholders() to add the brackets and underscore
df["clean_query"] = df["clean_query"].apply(replace_placeholders)
fallb_df["clean_fallback"] = fallb_df["clean_fallback"].apply(replace_placeholders)

In [29]:
df["clean_query"].tail()

7710                                         wie rijd mee
7711                                       rijd ik alleen
7712                               hoe werkt een proefrit
7713    ik wil graag en proefrit maken met de [CAR_MODEL]
7714                                          [CAR_BRAND]
Name: clean_query, dtype: object

### Removing stop words

In [30]:
# add custom words to the stopwords list
custom_stopwords = ["tijdschrift", "karcher", "graag", "tijdschriftnl", "jullie", "licechat", "grrrr"]

In [31]:
stops = set(stopwords.words("english"))
print(stops)
stops = set(stopwords.words("dutch"))
print(stops)

{'re', "wouldn't", 'ours', 'yourself', 'that', 'needn', 'my', 'herself', 'an', 'because', 'of', 'are', 'your', 'didn', 'hasn', "won't", 'been', 'won', 'hers', 'where', 'd', 'can', 'she', 'its', 'am', 'yours', 'he', "isn't", "wasn't", 'about', 'few', 'mustn', 'did', 'myself', 'our', "you'd", 'ain', "didn't", 'these', 'other', 'so', "she's", "mustn't", 'why', 'what', 'them', 'the', 'i', "that'll", 'both', 'this', 'more', 'does', 'up', "doesn't", 'me', 'then', "hasn't", 'again', 'him', 'whom', 'during', 'own', 'shan', 'with', 'too', 'which', 'who', 'when', 'and', 'very', "haven't", 'after', 'y', 'they', 'only', 'm', 'now', 'haven', 'has', 'most', 'but', "it's", 's', 'shouldn', 'their', "needn't", 'into', 'ourselves', 'his', 'themselves', 'isn', 'have', 'do', 'under', 'down', 'a', 'aren', 'couldn', "you're", "weren't", 'some', "hadn't", 'over', 'just', 'once', 't', 'yourselves', 'is', 'weren', 'than', 'above', 'were', 'itself', 'each', 've', 'was', 'no', 'her', 'in', "shan't", 'for', 'or',

In [32]:
def remove_stopwords(text):
  
  # load the default stopwords list from NLTK for Dutch, German and English and add custom stopwords
    stop_words = set(stopwords.words("dutch")).union(set(stopwords.words("english"))).union(set(stopwords.words("german"))).union(custom_stopwords)
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    
    if len(filtered_tokens) > 1:
        masked_text = " ".join(filtered_tokens)
        masked_text = masked_text.replace("[ ", "[")
        masked_text = masked_text.replace(" ]", "]")
        return masked_text
    else:
        return None

In [33]:
# apply the function to each text in your data
df["clean_query"] = df["clean_query"].apply(remove_stopwords)
# fallb_df["clean_fallback"] = fallb_df["clean_fallback"].apply(remove_stopwords)

In [34]:
df["query"][1112]

'[PRODUCT_BRAND] niet bezorgd'

In [35]:
df["clean_query"][1112]

'[PRODUCT_BRAND] bezorgd'

In [36]:
df["clean_query"].tail()

7710                      rijd mee
7711                   rijd alleen
7712                werkt proefrit
7713    proefrit maken [CAR_MODEL]
7714                   [CAR_BRAND]
Name: clean_query, dtype: object

In [37]:
# check for values, which are empty strings
empty_strings = df[df["clean_query"] == ""].index
print(empty_strings)

num_empty_strings = len(empty_strings)
print(num_empty_strings)

Int64Index([], dtype='int64')
0


In [38]:
# replace any empty strings in the "clean_query" column with np.nan objects
df["clean_query"].replace("", np.nan, inplace=True)

In [39]:
# drop the nan values
df.dropna(subset=["clean_query"], inplace=True)

> Export the cleaned data.

In [40]:
# clean_df = df.clean_query

In [41]:
# data = clean_df.to_csv("df_not_lemmatized.csv")

In [42]:
# apply the function to each text 
clean_data = df["clean_query"] 
clean_fallb = fallb_df["clean_fallback"]

clean_data_ = clean_fallb.to_csv("clean_query_data.csv")
clean_fallb_ = clean_fallb.to_csv("clean_query_fallback.csv")

### Lemmatize the data

In [43]:
nlp_nl = spacy.load("nl_core_news_md")
nlp_en = spacy.load("en_core_web_sm")
nlp_de = spacy.load("de_core_news_sm")

def lemmatize_text(text, lang="nl"):
    if lang == "nl":
        nlp = nlp_nl
    elif lang == "en":
        nlp = nlp_en
    elif lang == "de":
        nlp = nlp_de
    else:
        raise ValueError(f"Unsupported language: {lang}")

    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]    

    return " ".join(lemmatized_tokens)

In [44]:
# apply the function to the data
df["lemmatized_query"] = df["clean_query"].apply(lemmatize_text)

In [45]:
df["clean_query"][25]

'komende maand betalen abonnement stopgezet'

In [46]:
df["lemmatized_query"][25]

'komen maand betalen abonnement stopzetten'

In [47]:
print(lemmatize_text("komende maand betalen abonnement stopgezet"))

komen maand betalen abonnement stopzetten


In [48]:
df["lemmatized_query"].tail(30)

7679                                       regel proefrit
7681                                   proefrit aanvragen
7682                                 proefrit huis gratis
7684                                  proefrit bevestigen
7685                       proefrit [ car_model ] hybride
7686                                      proefrit zondag
7687                                 zondag proefrit doen
7689              waarom lukken proefrit plan troc cabrio
7690    dealer waal proefrit maken [ car_model ] waar ...
7691                               proefrit [ car_model ]
7692                     proefrit aangevraagd bevestiging
7693                                 bevestiging proefrit
7694                               auto kopen financieren
7695                                     auto financieren
7696                                        plan proefrit
7698                                vetkoper gaan dwingen
7699                                verkoper gaan dwingen
7700          

In [49]:
def replace_masks(text):
    text = re.sub(r"\[\s*product\_brand+\s*\]", "[PRODUCT_BRAND]", text)
    text = re.sub(r"\[\s*product\_model+\s*\]", "[PRODUCT_MODEL]", text)
    text = re.sub(r"\[\s*product\_part+\s*\]", "[PRODUCT_PART]", text)
    text = re.sub(r"\[\s*car\_brand+\s*\]", "[CAR_BRAND]", text)
    text = re.sub(r"\[\s*car\_model+\s*\]", "[CAR_MODEL]", text)
    text = re.sub(r"\[\s*mask\_phone+\s*\]", "[MASK_PHONE]", text)
    text = re.sub(r"\[\s*mask\_email+\s*\]", "[MASK_EMAIL]", text)
    
    return text

In [50]:
df["lemmatized_query"] = df["lemmatized_query"].apply(replace_masks)

In [51]:
df["lemmatized_query"][25]

'komen maand betalen abonnement stopzetten'

In [58]:
df["lemmatized_query"].tail(20)

7692                     proefrit aangevraagd bevestiging
7693                                 bevestiging proefrit
7694                               auto kopen financieren
7695                                     auto financieren
7696                                        plan proefrit
7698                                vetkoper gaan dwingen
7699                                verkoper gaan dwingen
7700                                     betalen proefrit
7701                               hoelang duren proefrit
7702    proefrit aangevraagd bijna uur reaktie hebben ...
7703                                steeds contact hebben
7704                                       auto opgehalen
7705                                         ophalen auto
7708                                          waar letten
7709                                          proefrit id
7710                                           rijden mee
7711                                        rijden alleen
7712          

In [52]:
dataset = df["lemmatized_query"]
dataset.to_csv("df_lemmatized_masks.csv")

> Look into the rows, which consisted of a number and have became empty strings after cleaning.

In [53]:
# check for values, which are empty strings
empty_strings = df[df["lemmatized_query"] == ""].index
print(empty_strings)

num_empty_strings = len(empty_strings)
print(num_empty_strings)

Int64Index([], dtype='int64')
0


In [54]:
# replace any empty strings in the "clean_query" column with np.nan objects
df["lemmatized_query"].replace("", np.nan, inplace=True)

In [55]:
# drop the nan values
df.dropna(subset=["lemmatized_query"], inplace=True)

In [56]:
# check for values, which are empty strings
empty_str = fallb_df[fallb_df["lemmatized_query"] == ""].index
print(empty_str)

num_empty_strings = len(empty_strings)
print(num_empty_strings)

KeyError: ignored

In [ ]:
# replace any empty strings in the "clean_fallback" column with np.nan objects
fallb_df["lemmatized_query"].replace("", np.nan, inplace=True)

In [ ]:
# drop the nan values
fallb_df.dropna(subset=["lemmatized_query"], inplace=True)

In [ ]:
df["clean_query"][30]

In [ ]:
df["query"][465]

In [ ]:
df["lemmatized_query"][465]

In [ ]:
# check which rows are starting with the below string
dupl = df[df["lemmatized_query"].str.contains("tijdschift")]

# select only the 'query' column to see the user's input
dupli = dupl[["query", "lemmatized_query"]]

# display the resulting DataFrame
dupli.head(50)

### Dealing with repetitive words

In [ ]:
# def lemmatize_text(text):
#     doc = nlp(text)
#     lemmatized_tokens = [token.lemma_.lower() for token in doc]

#     doc_nl = nlp(text)
#     lemmatized_tokens_nl = [token.lemma_.lower() for token in doc_nl]

#     return " ".join(lemmatized_tokens + lemmatized_tokens_nl)

In [ ]:
# # apply the function to each text in your data
# clean_data = df["clean_query"] 
# clean_fallb = fallb_df["clean_fallback"]

In [ ]:
# # apply the function to each text in your data
# clean_data = df["clean_query"] 
# clean_fallb = fallb_df["clean_fallback"]

# clean_data = clean_fallb.to_csv("clean_query_data.csv")
# clean_fallb = clean_fallb.to_csv("clean_query_fallback.csv")

In [ ]:
# # apply the function to each text in your data
# lemmatized_df = df["lemmatized_query"] 
# lemmatized_fallb = fallb_df["lemmatized_query"]

In [ ]:
# lemmatized_data = lemmatized_df.to_csv("lemmatized_df.csv")
# lemmatized_fallb = lemmatized_fallb.to_csv("lemmatized_fallbd.csv")